<a href="https://colab.research.google.com/github/currencyfxjle/Knapsack---GDF---3.0/blob/main/Knapsack_GDF_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread
!pip install gspread-dataframe
!pip install --upgrade oauth2client

In [2]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.1 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import os

# List files and folders under My Drive
base_path = '/content/drive/My Drive'
for root, dirs, files in os.walk(base_path):
    print(f"Directory: {root}")
    for directory in dirs:
        print(f"  Sub-directory: {directory}")
    for file in files:
        print(f"  File: {file}")

Directory: /content/drive/My Drive
  Sub-directory: Risk Modeling
  Sub-directory: Colab Notebooks
  File: Financial Concepts   Solar Energy Greenday Finance.gdoc
  File: config.zip
  File: 2024 Project Allocation Results (7).gsheet
  File: 2024 Project Allocation Results (6).gsheet
  File: 2024 Project Allocation Results (5).gsheet
  File: 2024 Project Allocation Results (4).gsheet
  File: 2024 Project Allocation Results (3).gsheet
  File: 2024 Project Allocation Results (2).gsheet
  File: 2024 Project Allocation Results (1).gsheet
  File: 2024 Project Allocation Results.gsheet
  File: Estados de Resultados - Victor Lizarraga.gdoc
  File: Ajustes  knapsack Lalo.gsheet
  File: Backup 2025.gsheet
  File: Ruta calculadora.gdoc
  File: KNAPSACK - GDF - Last Version 2.1.ipynb
Directory: /content/drive/My Drive/Risk Modeling
  File: Risk Model - Volatility of financial variables.ipynb
Directory: /content/drive/My Drive/Colab Notebooks
  Sub-directory: GDF - Knapsack
  File: GDF - Risk Model

In [5]:
import os

correct_path = '/content/drive/My Drive/Colab Notebooks/GDF - Knapsack/2025 Project Agenda.gsheet'

if os.path.exists(correct_path):
    print("File exists:", correct_path)
else:
    print("File does NOT exist:", correct_path)

File exists: /content/drive/My Drive/Colab Notebooks/GDF - Knapsack/2025 Project Agenda.gsheet


In [7]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['project_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')
    supply_df = supply_df.dropna()
    supply_df = supply_df.sort_values(by=['project_date'], ascending=True)

    # Save original Est Cash Amount and add a "Fully Allocated" flag
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Categorize Est Cash Amount into buckets
    supply_df['Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )
    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['historic allocated amount'] = pd.to_numeric(demand_df['allocated_amount'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['original_pending_amount'] = demand_df['updated pending amount']  # Save original pending amount
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df = demand_df.dropna()

    # Categorize Total Appetite into buckets
    demand_df['Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )
    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    total_allocated = 0
    allocations = []
    unallocated_demand = []
    project_tracking = []

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['updated pending amount']
        total_appetite = purchaser['approximated_appetite']

        for jdx, project in supply_df.iterrows():
            project_cost = project['Est Cash Amount']
            project_name = project['Proyect Name']
            project_id = project['Project ID']
            project_date = project['project_date']

            if project_cost > remaining_pending_amount:
                continue
            if project_cost > total_appetite:
                continue
            if project['Fully Allocated']:
                continue
            if project_date < purchaser['effective_date_purchaser']:
                continue

            allocations.append({
                "Purchaser": purchaser['purchaser_name'],
                "Project ID": project_id,
                "Project Name": project_name,
                "Original Est Cash Amount": project['Original Est Cash Amount'],
                "Allocated Amount": project_cost,
                "Total Appetite": total_appetite,
                "Remaining Appetite": total_appetite,  # Total appetite remains unchanged
                "Updated Pending Amount": remaining_pending_amount - project_cost,  # Pending amount updates
                "Effective Date Purchaser": purchaser['effective_date_purchaser'],
                "Project Date": project_date.date(),
                "Fully Allocated": "Yes" if project_cost == project['Original Est Cash Amount'] else "No",
                "Purchaser Category": purchaser['Category'],
                "Project Category": project['Category']
            })
            remaining_pending_amount -= project_cost
            total_allocated += project_cost
            supply_df.at[jdx, 'Est Cash Amount'] = 0
            supply_df.at[jdx, 'Fully Allocated'] = True

            project_tracking.append({
                "Project Name": project_name,
                "Total Chunks": len(supply_df[supply_df['Proyect Name'] == project_name]),
                "Chunks Acquired": len(supply_df[(supply_df['Proyect Name'] == project_name) & (supply_df['Fully Allocated'] == True)]),
                "Chunks Remaining": len(supply_df[(supply_df['Proyect Name'] == project_name) & (supply_df['Fully Allocated'] == False)]),
                "Acquired Amounts": project_cost,
                "Acquired By": purchaser['purchaser_name'],
                "Total Appetite": total_appetite,  # Remains unchanged
                "Updated Pending Amount": remaining_pending_amount,  # Updates dynamically
                "Project Category": project['Category']  # Add project category
            })

            if remaining_pending_amount <= 0:
                break

        demand_df.at[idx, 'updated pending amount'] = remaining_pending_amount

        if remaining_pending_amount > 0:
            unallocated_demand.append({
                "Purchaser Name": purchaser['purchaser_name'],
                "Approximated Appetite": purchaser['approximated_appetite'],
                "Historic Allocated Amount": purchaser['historic allocated amount'],
                "Updated Pending Amount": remaining_pending_amount,
                "Deal Stage": purchaser['deal_stage'],
                "State": purchaser['state'],
                "Effective Date Purchaser": purchaser['effective_date_purchaser']
            })

    return total_allocated, allocations, unallocated_demand, supply_df, project_tracking

# Iterative Allocation Process
def iterative_allocation(supply_df, demand_df, max_iterations=5):
    iteration = 0
    total_allocated = 0
    all_allocations = []
    project_tracking = []

    while iteration < max_iterations:
        iteration += 1
        current_allocated, allocations, unallocated_demand, remaining_supply, tracking = match_supply_to_demand(supply_df, demand_df)
        total_allocated += current_allocated
        all_allocations.extend(allocations)
        project_tracking.extend(tracking)

        if current_allocated == 0:
            break

        supply_df = remaining_supply[remaining_supply['Est Cash Amount'] > 0].copy()

    return total_allocated, all_allocations, supply_df, demand_df, project_tracking

# Save Results to Separate Tabs in Excel
def save_to_excel(allocations, summary, unallocated_demand, remaining_supply, project_tracking, filename):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)

        pd.DataFrame(summary).to_excel(writer, sheet_name="Summary", index=False)

        pd.DataFrame(unallocated_demand).to_excel(writer, sheet_name="Unallocated Demand", index=False)

        remaining_supply.to_excel(writer, sheet_name="Remaining Supply", index=False)

        pd.DataFrame(project_tracking).drop_duplicates().to_excel(writer, sheet_name="Project Tracking", index=False)

    print(f"Results saved to: {filename}")

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()
total_allocated, allocations, remaining_supply, remaining_demand, project_tracking = iterative_allocation(supply_df, demand_df)

# Calculations for Summary
total_project_value = supply_df['Original Est Cash Amount'].sum()
total_pending_amount = demand_df['updated pending amount'].sum() if not demand_df.empty else 0
remaining_supply_value = remaining_supply['Est Cash Amount'].sum()
total_unallocated_value_original = demand_df['original_pending_amount'].sum() if 'original_pending_amount' in demand_df else 0
total_unallocated_value_appetite = demand_df['approximated_appetite'].sum() if 'approximated_appetite' in demand_df else 0

# Summary Data
summary_data = pd.DataFrame([
    {"Metric": "Original Total Unallocated Value (Total Appetite)", "Value": total_unallocated_value_appetite},
    {"Metric": "Original Total Project Value (Est Cash Amount)", "Value": total_project_value},
    {"Metric": "Total Allocated", "Value": total_allocated},
    {"Metric": "Remaining Supply Value After", "Value": remaining_supply_value},
    {"Metric": "Total Pending Amount", "Value": total_pending_amount},
])

# Save Results to Excel with Tabs
save_to_excel(
    allocations,
    summary_data,
    remaining_demand,
    remaining_supply,
    project_tracking,
    "allocation_results.xlsx"
)

# Results Summary
print(f"Original Total Unallocated Value (Total Appetite): {total_unallocated_value_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {total_project_value}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")


Results saved to: allocation_results.xlsx
Original Total Unallocated Value (Total Appetite): 110880429.1
Original Total Project Value (Est Cash Amount): 44025757.750000015
Total Allocated: 32683960.300000012
Remaining Supply Value After: 11341797.450000003
Total Pending Amount: 78196468.8


In [8]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['project_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')
    supply_df = supply_df.dropna()
    supply_df = supply_df.sort_values(by=['project_date'], ascending=True)

    # Save original Est Cash Amount and add a "Fully Allocated" flag
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Categorize Est Cash Amount into buckets
    supply_df['Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )
    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['historic allocated amount'] = pd.to_numeric(demand_df['allocated_amount'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['original_pending_amount'] = demand_df['updated pending amount']  # Save original pending amount
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df = demand_df.dropna()

    # Categorize Total Appetite into buckets
    demand_df['Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )
    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    total_allocated = 0
    allocations = []
    unallocated_demand = []
    project_tracking = []

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['updated pending amount']
        total_appetite = purchaser['approximated_appetite']

        for jdx, project in supply_df.iterrows():
            project_cost = project['Est Cash Amount']
            project_name = project['Proyect Name']
            project_id = project['Project ID']
            project_date = project['project_date']

            # Log attempt details
            print(f"Trying to match Purchaser: {purchaser['purchaser_name']} with Project: {project_name}")
            print(f"Remaining Pending Amount: {remaining_pending_amount}, Total Appetite: {total_appetite}, Project Cost: {project_cost}")

            if project_cost > remaining_pending_amount:
                print(f"Skipping - Project cost ({project_cost}) exceeds remaining pending amount ({remaining_pending_amount}).")
                continue
            if project_cost > total_appetite:
                print(f"Skipping - Project cost ({project_cost}) exceeds total appetite ({total_appetite}).")
                continue
            if project['Fully Allocated']:
                print("Skipping - Project already fully allocated.")
                continue
            if project_date < purchaser['effective_date_purchaser']:
                print(f"Skipping - Project date ({project_date}) does not satisfy the purchaser's effective date ({purchaser['effective_date_purchaser']}).")
                continue

            # Allocate project
            print(f"Allocating Project: {project_name}, Purchaser: {purchaser['purchaser_name']}")
            allocations.append({
                "Purchaser": purchaser['purchaser_name'],
                "Project ID": project_id,
                "Project Name": project_name,
                "Original Est Cash Amount": project['Original Est Cash Amount'],
                "Allocated Amount": project_cost,
                "Total Appetite": total_appetite,
                "Remaining Appetite": total_appetite,  # Total appetite remains unchanged
                "Updated Pending Amount": remaining_pending_amount - project_cost,  # Pending amount updates
                "Effective Date Purchaser": purchaser['effective_date_purchaser'],
                "Project Date": project_date.date(),
                "Fully Allocated": "Yes" if project_cost == project['Original Est Cash Amount'] else "No",
                "Purchaser Category": purchaser['Category'],
                "Project Category": project['Category']
            })

            # Update allocation details
            remaining_pending_amount -= project_cost
            total_allocated += project_cost
            supply_df.at[jdx, 'Est Cash Amount'] = 0
            supply_df.at[jdx, 'Fully Allocated'] = True

            project_tracking.append({
                "Project Name": project_name,
                "Total Chunks": len(supply_df[supply_df['Proyect Name'] == project_name]),
                "Chunks Acquired": len(supply_df[(supply_df['Proyect Name'] == project_name) & (supply_df['Fully Allocated'] == True)]),
                "Chunks Remaining": len(supply_df[(supply_df['Proyect Name'] == project_name) & (supply_df['Fully Allocated'] == False)]),
                "Acquired Amounts": project_cost,
                "Acquired By": purchaser['purchaser_name'],
                "Total Appetite": total_appetite,  # Remains unchanged
                "Updated Pending Amount": remaining_pending_amount,  # Updates dynamically
                "Project Category": project['Category']  # Add project category
            })

            if remaining_pending_amount <= 0:
                break

        demand_df.at[idx, 'updated pending amount'] = remaining_pending_amount

        if remaining_pending_amount > 0:
            unallocated_demand.append({
                "Purchaser Name": purchaser['purchaser_name'],
                "Approximated Appetite": purchaser['approximated_appetite'],
                "Historic Allocated Amount": purchaser['historic allocated amount'],
                "Updated Pending Amount": remaining_pending_amount,
                "Deal Stage": purchaser['deal_stage'],
                "State": purchaser['state'],
                "Effective Date Purchaser": purchaser['effective_date_purchaser']
            })

    return total_allocated, allocations, unallocated_demand, supply_df, project_tracking

# Iterative Allocation Process
def iterative_allocation(supply_df, demand_df, max_iterations=5):
    iteration = 0
    total_allocated = 0
    all_allocations = []
    project_tracking = []

    while iteration < max_iterations:
        iteration += 1
        print(f"--- Iteration {iteration} ---")
        current_allocated, allocations, unallocated_demand, remaining_supply, tracking = match_supply_to_demand(supply_df, demand_df)
        print(f"Allocated in this iteration: {current_allocated}")
        print(f"Remaining Supply: {remaining_supply['Est Cash Amount'].sum()}")
        print(f"Unallocated Demand: {sum([d['Updated Pending Amount'] for d in unallocated_demand])}")
        total_allocated += current_allocated
        all_allocations.extend(allocations)
        project_tracking.extend(tracking)

        if current_allocated == 0:
            break

        supply_df = remaining_supply[remaining_supply['Est Cash Amount'] > 0].copy()

    return total_allocated, all_allocations, supply_df, demand_df, project_tracking

# Save Results to Separate Tabs in Excel
def save_to_excel(allocations, summary, unallocated_demand, remaining_supply, project_tracking, filename):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)

        pd.DataFrame(summary).to_excel(writer, sheet_name="Summary", index=False)

        pd.DataFrame(unallocated_demand).to_excel(writer, sheet_name="Unallocated Demand", index=False)

        remaining_supply.to_excel(writer, sheet_name="Remaining Supply", index=False)

        pd.DataFrame(project_tracking).drop_duplicates().to_excel(writer, sheet_name="Project Tracking", index=False)

    print(f"Results saved to: {filename}")

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()
total_allocated, allocations, remaining_supply, remaining_demand, project_tracking = iterative_allocation(supply_df, demand_df)

# Calculations for Summary
total_project_value = supply_df['Original Est Cash Amount'].sum()
total_pending_amount = demand_df['updated pending amount'].sum() if not demand_df.empty else 0
remaining_supply_value = remaining_supply['Est Cash Amount'].sum()
total_unallocated_value_original = demand_df['original_pending_amount'].sum() if 'original_pending_amount' in demand_df else 0
total_unallocated_value_appetite = demand_df['approximated_appetite'].sum() if 'approximated_appetite' in demand_df else 0

# Summary Data
summary_data = pd.DataFrame([
    {"Metric": "Original Total Project Value (Est Cash Amount)", "Value": total_project_value},
    {"Metric": "Total Allocated", "Value": total_allocated},
    {"Metric": "Remaining Supply Value After", "Value": remaining_supply_value},
    {"Metric": "Total Pending Amount", "Value": total_pending_amount},
    {"Metric": "Original Total Unallocated Value (Total Appetite)", "Value": total_unallocated_value_appetite}
])

# Save Results to Excel with Tabs
save_to_excel(
    allocations,
    summary_data,
    remaining_demand,
    remaining_supply,
    project_tracking,
    "allocation_results.xlsx"
)

# Results Summary
print(f"Original Total Project Value (Est Cash Amount): {total_project_value}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")
print(f"Original Total Unallocated Value (Total Appetite): {total_unallocated_value_appetite}")


Se truncaron las últimas líneas 5000 del resultado de transmisión.
Remaining Pending Amount: 500000.0, Total Appetite: 500000.0, Project Cost: 711228.35
Skipping - Project cost (711228.35) exceeds remaining pending amount (500000.0).
Trying to match Purchaser: Lu Yang & Jun Qiu with Project: Lewistone Solar II
Remaining Pending Amount: 500000.0, Total Appetite: 500000.0, Project Cost: 889035.44
Skipping - Project cost (889035.44) exceeds remaining pending amount (500000.0).
Trying to match Purchaser: Lu Yang & Jun Qiu with Project: Lewistone Solar II
Remaining Pending Amount: 500000.0, Total Appetite: 500000.0, Project Cost: 889035.44
Skipping - Project cost (889035.44) exceeds remaining pending amount (500000.0).
Trying to match Purchaser: Lu Yang & Jun Qiu with Project: Lewistone Solar II
Remaining Pending Amount: 500000.0, Total Appetite: 500000.0, Project Cost: 889035.44
Skipping - Project cost (889035.44) exceeds remaining pending amount (500000.0).
Trying to match Purchaser: Lu Y